In [99]:
import numpy as np 

# Given Functions

In [100]:
def ConvertToInt(message_str):
    res = 0
    for i in range(len(message_str)):
        res = res * 256 + ord(message_str[i])
    return res

def ConvertToStr(n):
    res = ""
    while n > 0:
        res += chr(n % 256)
        n //= 256
    return res[::-1]

def GCD(a, b):
    if b == 0:
        return a
    return GCD(b, a % b)

def ExtendedEuclid(a, b):
    if b == 0:
        return (1, 0)
    (x, y) = ExtendedEuclid(b, a % b)
    k = a // b
    return (y, x - k * y)

def InvertModulo(a, n):
    (b, x) = ExtendedEuclid(a, n)
    if b < 0:
        b = (b % n + n) % n # we don’t want −ve integers
    return b

def PowMod(a, n, mod):
    if n == 0:
        return 1 % mod
    elif n == 1:
        return a % mod
    else:
        b = PowMod(a, n // 2, mod)
        b = b * b % mod
        if n % 2 == 0:
            return b
        else:
            return b * a % mod

# 1. Encryption and Decryption Modules

In [101]:
def Encrypt(m, n, e):
    c = PowMod(ConvertToInt(m),e, n)
    return c
    
def Decrypt(c, p, q, e):
    phi = (p-1) * (q-1)
    d = InvertModulo(e , phi)
    m = ConvertToStr(PowMod( c , d , p*q))
    return m

In [102]:
p = 1000000007
q = 1000000009
exponent = 23917
modulo = p * q
ciphertext = Encrypt("attack", modulo, exponent)
message = Decrypt(ciphertext, p, q, exponent)
print(message)

attack


# 2. Simple Attack

In [103]:
def DecipherSimple(c, n, e, potential_messages):
    decipheredtext = ''
    for i in range(len(potential_messages)):
        c_potential_messages = Encrypt(potential_messages[i], n, e)
        if c == c_potential_messages:
            decipheredtext = potential_messages[i]
            break
    return decipheredtext

In [104]:
modulo = 101
exponent = 12
ciphertext = Encrypt("attack", modulo, exponent)
print(DecipherSimple(ciphertext, modulo, exponent, ["attack", "don't attack", "wait"]))

attack


# 3. Small Prime Attack

In [105]:
def DecipherSmallPrime(c, n, e):
    decipheredtext = ''
    if n % 2 == 0:
      decipheredtext = Decrypt(c, 2, n // 2, e) 
    else :
      for i in range(3, 10**6, 2):
        if n % i == 0:
          decipheredtext = Decrypt(c, i, n // i, e)  
          break         
    return decipheredtext

In [106]:
modulo = 101 * 18298970732541109011012304219376080251334480295537316123696052970419466495220522723330315111017831737980079504337868198011077274303193766040393009648852841770668239779097280026631944319501437547002412556176186750790476901358334138818777298389724049250700606462316428106882097210008142941838672676714188593227684360287806974345181893018133710957167334490627178666071809992955566020058374505477745993383434501768887090900283569055646901291270870833498474402084748161755197005050874785474707550376333429671113753137201128897550014524209754619355308207537703754006699795711188492048286436285518105948050401762394690148387
exponent = 239
ciphertext = Encrypt("attack", modulo, exponent)
print(DecipherSmallPrime(ciphertext, modulo, exponent))

attack


# 4. Small Difference Attack

In [107]:
def DecipherSmallDiff(c, n, e):
    decipheredtext = ''    
    sqrtN = int(round(np.sqrt(float(n))))
    for i in range(sqrtN - 5000 , sqrtN ):
        if n % i == 0:
          decipheredtext = Decrypt(c, i, n // i, e)  
          break
    return decipheredtext

In [108]:
p = 1000000007
q = 1000000009
n = p * q
e = 239
ciphertext = Encrypt("attack", n, e)
message = DecipherSmallDiff(ciphertext, n, e)
print(message)

attack


# 5. Common Divisor Attack

In [109]:
def DecipherCommonDivisor(c1, n1, e1, c2, n2, e2):
    p = GCD(n1 , n2)
    q1 = n1 // p
    q2 = n2 // p 
    first_decipheredtext  = Decrypt(c1 , p , q1 , e1)
    second_decipheredtext = Decrypt(c2 , p , q2 , e2)    
    return first_decipheredtext, second_decipheredtext

In [110]:
p = 101
q1 = 18298970732541109011012304219376080251334480295537316123696052970419466495220522723330315111017831737980079504337868198011077274303193766040393009648852841770668239779097280026631944319501437547002412556176186750790476901358334138818777298389724049250700606462316428106882097210008142941838672676714188593227684360287806974345181893018133710957167334490627178666071809992955566020058374505477745993383434501768887090900283569055646901291270870833498474402084748161755197005050874785474707550376333429671113753137201128897550014524209754619355308207537703754006699795711188492048286436285518105948050401762394690148387
q2 = 1000000007
first_modulo = p * q1
second_modulo = p * q2
first_exponent = 239
second_exponent = 17
first_ciphertext = Encrypt("attack", first_modulo, first_exponent)
second_ciphertext = Encrypt("wait", second_modulo, second_exponent)
print(DecipherCommonDivisor(first_ciphertext, first_modulo, first_exponent, second_ciphertext, second_modulo, second_exponent))

('attack', 'wait')


# 6. Hastad’s Attack

In [111]:
def DecipherHastad(c1, n1, c2, n2, e):
    broadcastmessage = ''
    n = n1 * n2
    N1 = n // n1
    N2 = n // n2
    x1 = InvertModulo(N1 , n1)
    x2 = InvertModulo(N2 , n2)
    c = (c1 * N1 * x1 + c2 * N2 * x2 )%n
    m = int(round(np.power(c , 1./e)))
    broadcastmessage = (ConvertToStr(m))
    return broadcastmessage

In [112]:
p1 = 790383132652258876190399065097
q1 = 662503581792812531719955475509
p2 = 656917682542437675078478868539
q2 = 1263581691331332127259083713503
n1 = p1 * q1
n2 = p2 * q2
e = 2
ciphertext1 = Encrypt("attack", n1, e)
ciphertext2 = Encrypt("attack", n2, e)
message = DecipherHastad(ciphertext1, n1, ciphertext2, n2, e)
print(message)

attack
